In [62]:
import psycopg2
from sqlalchemy import create_engine

import pandas as pd
import numpy as np
import string

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, roc_curve, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer


from create_staging_tables import create_loyalty_scores_df
from edit_current_legislator import filter_out_duplicates_from_current_leg
from clean_for_model import prepare_bill_text_for_clean_df

import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

In [39]:
engine = create_engine('postgresql://localhost:5432/wa_leg_staging')

In [77]:
merged_final_df = pd.read_sql_query('select * from "merged_final"',con=engine)
bill_text_df_dirty = pd.read_sql_query('select * from "bill_text"',con=engine)
rep_score_df = pd.read_sql_query('select * from "rep_score"',con=engine)
loyalty_df = pd.read_sql_query('select * from "loyalty"',con=engine)

current_df = pd.read_sql_query('select * from "current_clean"',con=engine)
current_bill_text_dirty = pd.read_sql_query('select * from "current_bill_text"',con=engine)
current_legislator_df = pd.read_sql_query('select * from "current_legislator"',con=engine)

In [63]:
bill_text_df = prepare_bill_text_for_clean_df(bill_text_df_dirty)

In [41]:
loyalty_df.head()

,loyalty_score,percent_yea,voter_id
0,0.177242,0.952786,347
1,0.140579,1.060844,360
2,-0.182734,0.922963,361
3,-0.066394,1.114206,387
4,0.083411,0.928034,512


In [84]:
def create_clean_train(merged_final_df, bill_text_df, rep_score_df, loyalty_df):
    
    # Add bill_text
    clean = merged_final_df.merge(bill_text_df, how='left', on=['unique_id', 'htm_url'])
    clean.drop(['index_x', 'index_y'], axis=1, inplace=True)
    clean.rename(columns={'sponsor_party': 'primary_sponsor_party'}, inplace=True)

    
    # Add rep_score and replace -1 values, then make dem_score
    clean['bill_num'] = clean['bill_id'].apply(lambda x: x.split()[1])
    clean['bill_num_unique'] = clean['biennium'] + ' ' + clean['bill_num']
    clean = clean.merge(rep_score_df, how='left', on='bill_num_unique')

    def replace_with_p_sponsor_party(row):
        primary_sponsor_party = row['primary_sponsor_party']
        if row['rep_score'] == -1:
            return primary_sponsor_party
        else:
            return row['rep_score']
    clean['rep_score'] = clean.apply(replace_with_p_sponsor_party, axis=1)
    clean['dem_score'] = 1 - clean['rep_score']
    
    
#     Add loyalty scores
    clean = clean.merge(loyalty_df, how = 'left', on='voter_id')
    clean.drop('index', axis=1, inplace=True)
    
    
    # Feature Engineering functions
    def create_congress_nums(biennium):
        '''Numbers bienniums so that 1991 is 1 and 1993 is 3.'''
        first_year = int((biennium.split('-'))[0])
        return first_year - 1990
    
    def find_num_sponsors(secondary_sponsors):
        if type(secondary_sponsors) == str:
            return round((len(secondary_sponsors) / 5) + 1)
        else: return 1
        
    def remove_neg(num):
        if num < 0:
            return 0
        else: return num
        
    def count_sections(bill_text):
        sections = []
        for count in range(1, 20):
            section_indicator = '({})'.format(count)
            if section_indicator in bill_text:
                sections.append(count)
        if len(sections) == 0:
            return 0
        return np.max(sections)
    
    def get_bill_type(bill_id):
        bill_type = (bill_id.split(' '))[0]
        return bill_type
    
    # Feature Engineering
    clean['bill_length'] = clean['bill_text'].apply(lambda x: len(x))
    clean['num_sections'] = clean['bill_text'].apply(count_sections)
    clean['congress_num'] = clean['biennium'].apply(create_congress_nums)
    clean['num_sponsors'] = clean['secondary_sponsors'].apply(find_num_sponsors)
    
    clean['vote_year'] = clean['vote_date'].apply(lambda x: x.year)
    clean['first_year_read'] = clean['first_date_read'].apply(lambda x: x.year)
    clean['first_year_read'] = clean['first_year_read'].fillna(clean['vote_year'])
    clean['years_until_vote'] = clean['vote_year'] - clean['first_year_read']
    clean['years_until_vote'] = clean['years_until_vote'].apply(remove_neg)
    
    clean['bill_type'] = clean['bill_id'].apply(get_bill_type)
    grouped = clean.groupby('bill_type').mean()
    bill_type_dct = grouped['vote'].to_dict()
    clean['bill_type_score'] = clean['bill_type'].apply(lambda x: bill_type_dct[x])

    return clean, bill_type_dct

In [85]:
train_clean, bill_type_dct = create_clean_train(merged_final_df, bill_text_df, rep_score_df, loyalty_df)

In [86]:
train_clean

,vote,vote_date,voter_id,voting_agency,bill_unique,unique_id,biennium,bill_id,htm_url,sponsor_agency,...,percent_yea,bill_length,num_sections,congress_num,num_sponsors,vote_year,first_year_read,years_until_vote,bill_type,bill_type_score
0,1,1993-02-16,347,1,1993-94 SB 5426,12053.0,1993-94,SB 5426,http://app.leg.wa.gov/documents/billdocs/1993-...,1,...,0.952786,33244,9,3,4,1993,1993.0,0.0,SB,0.980216
1,1,1993-02-16,360,1,1993-94 SB 5426,12053.0,1993-94,SB 5426,http://app.leg.wa.gov/documents/billdocs/1993-...,1,...,1.060844,33244,9,3,4,1993,1993.0,0.0,SB,0.980216
2,1,1993-02-16,361,1,1993-94 SB 5426,12053.0,1993-94,SB 5426,http://app.leg.wa.gov/documents/billdocs/1993-...,1,...,0.922963,33244,9,3,4,1993,1993.0,0.0,SB,0.980216
3,1,1993-02-16,387,1,1993-94 SB 5426,12053.0,1993-94,SB 5426,http://app.leg.wa.gov/documents/billdocs/1993-...,1,...,1.114206,33244,9,3,4,1993,1993.0,0.0,SB,0.980216
4,1,1993-02-16,512,1,1993-94 SB 5426,12053.0,1993-94,SB 5426,http://app.leg.wa.gov/documents/billdocs/1993-...,1,...,0.928034,33244,9,3,4,1993,1993.0,0.0,SB,0.980216
5,1,1993-02-16,399,1,1993-94 SB 5426,12053.0,1993-94,SB 5426,http://app.leg.wa.gov/documents/billdocs/1993-...,1,...,0.985877,33244,9,3,4,1993,1993.0,0.0,SB,0.980216
6,1,1993-02-16,402,1,1993-94 SB 5426,12053.0,1993-94,SB 5426,http://app.leg.wa.gov/documents/billdocs/1993-...,1,...,0.974854,33244,9,3,4,1993,1993.0,0.0,SB,0.980216
7,1,1993-02-16,403,1,1993-94 SB 5426,12053.0,1993-94,SB 5426,http://app.leg.wa.gov/documents/billdocs/1993-...,1,...,0.994833,33244,9,3,4,1993,1993.0,0.0,SB,0.980216
8,1,1993-02-16,405,1,1993-94 SB 5426,12053.0,1993-94,SB 5426,http://app.leg.wa.gov/documents/billdocs/1993-...,1,...,1.006436,33244,9,3,4,1993,1993.0,0.0,SB,0.980216
9,1,1993-02-16,409,1,1993-94 SB 5426,12053.0,1993-94,SB 5426,http://app.leg.wa.gov/documents/billdocs/1993-...,1,...,0.931835,33244,9,3,4,1993,1993.0,0.0,SB,0.980216


In [87]:
current_bill_text_df = prepare_bill_text_for_clean_df(current_bill_text_dirty)

In [88]:
def create_clean_test(current_df, current_bill_text_df, loyalty_df, bill_type_dct):
    
    # Add bill_text
    clean = current_df.merge(current_bill_text_df, how='left', on=['bill_id', 'biennium', 'htm_url'])
    clean.rename(columns={'sponsor_party': 'primary_sponsor_party'}, inplace=True)

    
    # Make dem_score
    clean['dem_score'] = 1 - clean['rep_score']
    
    
#   Add loyalty scores
    clean = clean.merge(loyalty_df, how = 'left', on='voter_id')
    
    # Feature Engineering functions
    def create_congress_nums(biennium):
        '''Numbers bienniums so that 1991 is 1 and 1993 is 3.'''
        first_year = int((biennium.split('-'))[0])
        return first_year - 1990
    
    def find_num_sponsors(secondary_sponsors):
        if type(secondary_sponsors) == str:
            return round((len(secondary_sponsors) / 5) + 1)
        else: return 1
        
    def remove_neg(num):
        if num < 0:
            return 0
        else: return num
        
    def count_sections(bill_text):
        sections = []
        for count in range(1, 20):
            section_indicator = '({})'.format(count)
            if section_indicator in bill_text:
                sections.append(count)
        if len(sections) == 0:
            return 0
        return np.max(sections)
    
    def get_bill_type(bill_id):
        bill_type = (bill_id.split(' '))[0]
        return bill_type
    
    # Feature Engineering
    clean['bill_length'] = clean['bill_text'].apply(lambda x: len(x))
    clean['num_sections'] = clean['bill_text'].apply(count_sections)
    clean['congress_num'] = clean['biennium'].apply(create_congress_nums)

    clean['vote_year'] = 2018
    clean['first_year_read'] = clean['first_date_read'].apply(lambda x: x.year)
    clean['first_year_read'] = clean['first_year_read'].fillna(clean['vote_year'])
    clean['years_until_vote'] = clean['vote_year'] - clean['first_year_read']
    clean['years_until_vote'] = clean['years_until_vote'].apply(remove_neg)
    
    # Filter out duplicate legislators
    clean['current'] = clean.apply(filter_out_duplicates_from_current_leg, axis=1)
    clean = clean[clean['current'] == True]
    
    clean['bill_type'] = clean['bill_id'].apply(get_bill_type)
    clean['bill_type_score'] = clean['bill_type'].apply(lambda x: bill_type_dct[x])

    return clean

In [89]:
test_clean = create_clean_test(current_df, current_bill_text_df, loyalty_df, bill_type_dct)

In [82]:
test_clean.columns

Index(['biennium', 'bill_id', 'bill_num', 'bill_unique', 'current', 'district',
       'first_name', 'htm_create_date', 'htm_url', 'last_name',
       'long_friendly_name', 'name', 'party', 'primary_sponsor_id',
       'secondary_sponsors', 'sponsor_agency', 'voter_id', 'voting_agency',
       'is_primary_sponsor', 'num_sponsors', 'is_minority_party',
       'is_secondary_sponsor', 'primary_sponsor_party', 'rep_score',
       'bill_text', 'first_date_read', 'dem_score', 'loyalty_score',
       'percent_yea', 'bill_length', 'congress_num', 'vote_year',
       'first_year_read', 'years_until_vote', 'bill_type', 'bill_type_score'],
      dtype='object')

### Current Top Model

In [90]:
# TRAIN
X_train_t = train_clean[['voter_id', 'voting_agency', 'sponsor_agency', 'district', 
                     'party', 'is_primary_sponsor', 'is_secondary_sponsor', 'is_minority_party', 
                     'primary_sponsor_party', 'rep_score', 'loyalty_score', 'bill_length', 'bill_num', 
                     'num_sponsors', 'years_until_vote', 'percent_yea', 'num_sections', 'bill_type_score']]

y_train_t = train_clean['vote']

# TEST
X_test_t = test_clean[['voter_id', 'voting_agency', 'sponsor_agency', 'district', 
                     'party', 'is_primary_sponsor', 'is_secondary_sponsor', 'is_minority_party', 
                     'primary_sponsor_party', 'rep_score', 'loyalty_score', 'bill_length', 'bill_num', 
                     'num_sponsors', 'years_until_vote', 'percent_yea', 'num_sections', 'bill_type_score']]

# y_test_t = sample_test['vote']

In [91]:
top_model = RandomForestClassifier(n_estimators=1000, max_depth=11, n_jobs=2, random_state=709)
top_model.fit(X_train_t, y_train_t)
y_pred_t = (top_model.predict_proba(X_test_t))[:, 1]
% time

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 9.78 µs


In [92]:
y_pred_t.min()

0.10278695233763821

In [93]:
test_clean['predicted_vote'] = y_pred_t

In [94]:
label_df = test_clean[['bill_id', 'bill_num', 'biennium', 'voter_id', 'voting_agency', 'predicted_vote', 
                       'rep_score', 'htm_url', 'secondary_sponsors']]

In [95]:
label_df = label_df.merge(current_legislator_df, how='left', on=['voter_id', 'voting_agency'])

In [96]:
label_df = label_df[label_df['last_name'].notnull()]

In [97]:
label_df[label_df['bill_num'] == '2299']

,bill_id,bill_num,biennium,voter_id,voting_agency,predicted_vote,rep_score,htm_url,secondary_sponsors,district,first_name,party,last_name,current
149949,ESHB 2299,2299,2017-18,10031,0,0.175002,0.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,23,Sherry,0,Appleton,True
149950,ESHB 2299,2299,2017-18,24075,0,0.828359,0.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,2,Andrew,1,Barkis,True
149951,ESHB 2299,2299,2017-18,17227,0,0.121127,0.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,11,Steve,0,Bergquist,True
149952,ESHB 2299,2299,2017-18,8317,0,0.175905,0.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,19,Brian,0,Blake,True
149953,ESHB 2299,2299,2017-18,15820,0,0.844583,0.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,42,Vincent,1,Buys,True
149954,ESHB 2299,2299,2017-18,20760,0,0.831609,0.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,26,Michelle,1,Caldier,True
149955,ESHB 2299,2299,2017-18,3469,0,0.795112,0.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,15,Bruce,1,Chandler,True
149956,ESHB 2299,2299,2017-18,26176,0,0.104530,0.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,24,Mike,0,Chapman,True
149957,ESHB 2299,2299,2017-18,1659,0,0.190656,0.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,43,Frank,0,Chopp,True
149958,ESHB 2299,2299,2017-18,8209,0,0.175067,0.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,41,Judy,0,Clibborn,True


In [98]:
engine_label = create_engine('postgresql://localhost:5432/wa_leg_label')

In [99]:
con = engine_label.connect()

In [100]:
label_df.to_sql('label_second', con, if_exists='replace', index=False)

In [ ]:
len(label_df)

In [ ]:
engine = create_engine('postgresql://localhost:5432/wa_leg_label')

In [25]:
labels = pd.read_sql_query('select * from "label"',con=engine_label)

In [26]:
label_bill_example = labels[labels['bill_num'] == '8408']

In [27]:
sorted_label = label_bill_example.sort_values('last_name')

In [29]:
sorted_label

,bill_id,bill_num,biennium,voter_id,voting_agency,predicted_vote,rep_score,htm_url,secondary_sponsors,district,first_name,party,last_name,current
445070,SCR 8408,8408,2017-18,14208,1,0.878938,1.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,26,Jan,1,Angel,True
444978,SCR 8408,8408,2017-18,10031,0,0.883090,1.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,23,Sherry,0,Appleton,True
445071,SCR 8408,8408,2017-18,8238,1,0.878891,1.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,10,Barbara,1,Bailey,True
444979,SCR 8408,8408,2017-18,24075,0,0.877445,1.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,2,Andrew,1,Barkis,True
445072,SCR 8408,8408,2017-18,15780,1,0.880005,1.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,6,Michael,1,Baumgartner,True
445073,SCR 8408,8408,2017-18,14083,1,0.876378,1.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,2,Randi,1,Becker,True
444980,SCR 8408,8408,2017-18,17227,0,0.880274,1.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,11,Steve,0,Bergquist,True
445074,SCR 8408,8408,2017-18,15811,1,0.836074,1.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,3,Andy,0,Billig,True
444981,SCR 8408,8408,2017-18,8317,0,0.882459,1.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,19,Brian,0,Blake,True
445075,SCR 8408,8408,2017-18,17289,1,0.878721,1.0,http://app.leg.wa.gov/documents/billdocs/2017-...,,20,John,1,Braun,True


In [ ]:
engine = create_engine('postgresql://localhost:5432/wa_leg_staging')

In [ ]:
legislator_df = pd.read_sql_query('select * from "legislator"',con=engine)

In [ ]:
legislator_df[legislator_df['id'] == 17279]

In [ ]:
current_legislator_df[current_legislator_df['voter_id'] == 17279]